# Jupyter Notebook: AWS Bedrock with LlamaIndex, LiamaParse and Elasticsearch

This notebook sets up a project that integrates **AWS Bedrock**, **LlamaIndex**, **LlamaParse** and **Elasticsearch** for building a retrieval-augmented generation (RAG) application. The notebook provides step-by-step guidance on installation, setup, and implementation.

## 1. Clone the Repository

```sh
git clone https://github.com/GenMindHub/07-LiamaIndex.git
cd 07-LiamaIndex
```

## 2. Create and Activate Virtual Environment

```sh
python3 -m venv .llamaindex
source .llamaindex/bin/activate
```

## 3. Install Jupyter Lab (Optional) 

If Jupyter Lab is not installed on your system, [install](https://jupyter.org/install) it using the following command:

```sh
python3 -m pip install -qU jupyterlab
python3 -m pip install -qU notebook
jupyter notebook password
jupyter lab
```

 ## 4. Install and Setup Elasticsearch

Follow the instructions [here](https://www.elastic.co/guide/en/elasticsearch/reference/current/run-elasticsearch-locally.html) to install and run Elasticsearch locally.

## 5. Install AWS CLI and Configure IAM Credentials

Install AWS CLI using the guide [here](https://docs.aws.amazon.com/cli/latest/userguide/getting-started-install.html).

Configure your AWS IAM credentials:

```sh
aws configure
```

## 6. Configure LlamaCloud API Key

Get your LlamaCloud API key. You can get one for free by [signing up](https://cloud.llamaindex.ai/) for LlamaCloud. Then put it in your .env file.

# Python Code Implementation

## Install Required Libraries

Explanation
- `boto3` : it is an official AWS SDK for Python that allows you to interact with AWS services programmatically. 
- `python-dotenv` : Allows you to load environment variables from a .env file.Useful for storing API keys, AWS credentials, and other sensitive data without hardcoding them in your script.
- `llama-index`: A framework for indexing and querying large-scale data using LLMs.Used for building retrieval-augmented generation (RAG) applications.
- `llama-index-vector-stores-elasticsearch`: Elasticsearch vector store integration for LlamaIndex.Allows you to store and retrieve embeddings efficiently using Elasticsearch.
- `llama-index-embeddings-bedrock`: AWS Bedrock embeddings integration for LlamaIndex.
Enables you to generate text embeddings using AWS Bedrock foundation models.
- `llama-index-llms-bedrock`: AWS Bedrock LLM integration for LlamaIndex.
Allows you to use AWS-hosted foundation models for answering queries and processing text.
- `nest_asyncio`: RuntimeError: As part of fix this error if you are running llamaparse load_document function in Jupyter - "Detected nested async. Please use nest_asyncio.apply() to allow nested event loops.Or, use async entry methods like aquery(), aretriever, achat, etc". Jupyter runs an internal event loop, and when you try to run another async function inside it, Python throws this error. To fix this, install and apply nest_asyncio at the beginning of your notebook.

In [8]:
%pip install boto3
%pip install python-dotenv
%pip install llama-index
%pip install llama-index-vector-stores-elasticsearch
%pip install llama-index-embeddings-bedrock
%pip install llama-index-llms-bedrock
%pip install nest_asyncio


[notice] A new release of pip is available: 24.2 -> 25.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.2 -> 25.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.2 -> 25.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.2 -> 25.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.2 -> 25.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.2 -> 25.0
[notice] To update, run: pip install --upgrade pip
Note: you may ne

## Import Required Libraries

In [9]:
import os
# Creates a connection with AWS services
import boto3
# Loads AWS credentials and other environment variables from a .env file.
from dotenv import load_dotenv

# Defines global configurations for LlamaIndex, such as embedding models and storage options.
from llama_index.core import Settings
# Manages persistent storage for vector embeddings and index metadata.
from llama_index.core import StorageContext
# Creates an index for storing document embeddings to enable efficient search
from llama_index.core import VectorStoreIndex

# Uses AWS Bedrock's foundation models (like Titan or Claude) to generate text embeddings.
from llama_index.embeddings.bedrock import BedrockEmbedding, Models
# Stores and retrieves document embeddings using Elasticsearch as a vector database.
from llama_index.vector_stores.elasticsearch import ElasticsearchStore

# Uses AWS Bedrock's LLMs for answering queries and processing text
from llama_index.llms.bedrock import Bedrock

# parsing documents
from llama_parse import LlamaParse

import nest_asyncio
nest_asyncio.apply()

## Function Definitions

### 1. Load and Parse PDF Documents

In [3]:
def load_documents_llama_parse(pdf_folder: str):
    documents = LlamaParse(result_type="markdown").load_data(pdf_folder)
    return documents

### 2. Initialize Elasticsearch as Vector Store

In [4]:
def initialize_elasticsearch():
    host = os.getenv("ELASTIC_HOST")
    username = os.getenv("ELASTIC_USERNAME")
    password = os.getenv("ELASTIC_PASSWORD")
    index_name = os.getenv("INDEX_NAME")
    
    vector_store = ElasticsearchStore(
        index_name=index_name, es_url=host, es_user=username, es_password=password
    )

    return vector_store

### 3. Embed Documents and Store in Vector Store

In [5]:
def create_index(documents, vector_store):
    
    # Initialize the bedrock embedding model 
    boto3_bedrock_client = boto3.client(service_name="bedrock-runtime")
    bedrock_embedding_model = BedrockEmbedding(model_name=os.getenv("AWS_BEDROCK_EMBEDDING_MODEL"), client=boto3_bedrock_client,)
    # Set Embeddings Globally Using `settings`
    Settings.embed_model = bedrock_embedding_model
    # assign OpenSearch as the vector_store to the context
    storage_context = StorageContext.from_defaults(vector_store=vector_store)
    # create index
    index = VectorStoreIndex.from_documents(documents, storage_context=storage_context, show_progress=True)

    return index

## Execution Flow

In [10]:
load_dotenv()

Settings.llm = Bedrock(model=os.getenv("AWS_BEDROCK_LLM_MODEL"), temperature=0, max_tokens=1000)

print("Loading documents...")
documents = load_documents_llama_parse("./data/2023_canadian_budget.pdf") # path is relative to virtual env
print(documents)

print("Initializing OpenSearch...")
vector_store = initialize_elasticsearch()
print(vector_store)

print("Creating index and storing embeddings...")
index = create_index(documents, vector_store)

query_engine = index.as_query_engine()
response = query_engine.query(
    "How much exactly was allocated to a tax credit to promote investment in green technologies in the 2023 Canadian federal budget?"
)
print(response)


Loading documents...
Started parsing the file under job_id e4ed1427-3d2f-4b10-aa47-cc3024fe0439
[Document(id_='35e7488e-1514-4661-b513-076d9cdd5f03', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text_resource=MediaResource(embeddings=None, data=None, text='# 2023 Canadian federal budget\n\nThe Canadian federal budget for the fiscal years of 2023–24 was presented to the House of Commons by Finance Minister Chrystia Freeland on 28 March 2023.2 The budget was meant to reflect Prime Minister Justin Trudeau\'s stated policy objective to "make life more affordable for Canadians"3 while also reducing government expenditures.4\n\n# Background\n\nThe 2023 budget is the seventh budget document introduced in the House of Commons under the premiership of Justin Trudeau. It comes at the heel of the first anniversary of the Russian invasion of Ukraine, following which Canada

/Users/mycloudjourney/Desktop/AI & ML/GitHub/GenMindHub/07-LiamaIndex/.llamaindex/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating embeddings: 100%|███████████████████████████████████████████████████████████████████████████████| 4/4 [00:07<00:00,  1.96s/it]


The 2023 Canadian federal budget included a new green investments tax credit, but the exact amount allocated to this tax credit is not specified in the given context information. The budget did include investments in renewable energy and green projects, but precise figures for the tax credit are not provided. Environmental organizations and some political leaders expressed concerns about how these credits would be used and their potential impact, but the specific monetary allocation for the green technologies tax credit is not mentioned in the provided information.


# Summary of workflow
- Load environment variables.
- Initialize AWS client for Bedrock.
- Set up LlamaIndex settings (embedding model, storage).
- Read pdf from a directory using Llama Parse. Click this [link](https://docs.llamaindex.ai/en/stable/llama_cloud/llama_parse/) to know more about LlamaParse.
- Create vector embeddings using AWS Bedrock.
- Store embeddings in Elasticsearch for retrieval.
- Set up Bedrock LLM for answering queries.
- Use query engine to respond to user queries dynamically.